# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [37]:
# 1) Imports + environment + model configuration

import os
import requests
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)

OPENROUTER_API_KEY = os.getenv("OPENAI_API_KEY")
OLLAMA_BASE_URL = "http://localhost:11434/v1"

openrouter_client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:7860",
        "X-Title": "week2_exercise_book_summarizer",
    },
)

ollama_client = OpenAI(
    base_url=OLLAMA_BASE_URL,
    api_key="ollama",
    timeout=90,
)

MODEL_OPTIONS = {
    "OpenRouter: gpt-4.1-mini": {
        "client": openrouter_client,
        "model": "gpt-4.1-mini",
    },
    "Ollama: llama3.2": {
        "client": ollama_client,
        "model": "llama3.2",
    },
}

SYSTEM_PROMPT = """
You are a precise book-summary assistant.

Rules:
- Use ONLY the provided public description, title, date published, and authors.
- Do NOT add facts not present in the description.
- Keep output concise and practical.
- Keep total output under 170 words.

Exact output format:
What it's about: <one sentence>
Author(s): <authors>
Date published: <date>

Key ideas:
- <idea 1>
- <idea 2>
- <idea 3>
- <idea 4>
- <idea 5>

Who it's for: <one line>
""".strip()

## 2) Google Books lookup (data source)

In [38]:
def lookup_book_google(title: str, author: str, max_results: int = 3):
    """Fetch best matching book metadata from Google Books."""
    query = f"intitle:{title} inauthor:{author}".strip()

    url = "https://www.googleapis.com/books/v1/volumes"
    params = {"q": query, "maxResults": max_results}

    response = requests.get(url, params=params, timeout=20)
    response.raise_for_status()
    data = response.json()

    items = data.get("items", [])
    if not items:
        return None

    best = items[0].get("volumeInfo", {})
    image_links = best.get("imageLinks", {}) or {}
    thumbnail = image_links.get("thumbnail") or image_links.get("smallThumbnail") or ""
    if isinstance(thumbnail, str) and thumbnail.startswith("http://"):
        thumbnail = "https://" + thumbnail[len("http://"):]

    return {
        "title": best.get("title", "Unknown"),
        "authors": best.get("authors", []),
        "publishedDate": best.get("publishedDate", "Unknown"),
        "categories": best.get("categories", []),
        "description": best.get("description", ""),
        "pageCount": best.get("pageCount"),
        "previewLink": best.get("previewLink", ""),
        "thumbnail": thumbnail,
    }


def build_user_prompt(meta: dict) -> str:
    authors = ", ".join(meta.get("authors") or []) or "Unknown"
    return f"""
Title: {meta.get('title', 'Unknown')}
Author(s): {authors}
Date published: {meta.get('publishedDate', 'Unknown')}

Public description:
{meta.get('description', '')}

Now follow the exact output format from the system message.
Do not add facts that are not in the public description.
""".strip()


def format_meta(meta: dict) -> str:
    authors = ", ".join(meta.get("authors") or []) or "Unknown"
    categories = ", ".join(meta.get("categories") or []) or "Unknown"
    pages = meta.get("pageCount") if meta.get("pageCount") else "Unknown"
    preview = meta.get("previewLink") or "N/A"

    return (
        f"**Title:** {meta.get('title', 'Unknown')}\n\n"
        f"**Author(s):** {authors}\n\n"
        f"**Published:** {meta.get('publishedDate', 'Unknown')}\n\n"
        f"**Categories:** {categories}\n\n"
        f"**Pages:** {pages}\n\n"
        f"**Preview:** {preview}"
    )

In [39]:
# 3) Summarization logic (streaming)

def summarize_book_stream(title: str, author: str, model_choice: str):
    title = (title or "").strip()
    author = (author or "").strip()

    if not title:
        yield "Please enter a book title.", "", None
        return

    if not author:
        yield "Please enter an author name.", "", None
        return

    meta = lookup_book_google(title=title, author=author, max_results=3)
    if not meta:
        yield "No matching book found in Google Books for that title/author.", "", None
        return

    cover_image = meta.get("thumbnail") or None

    if not meta.get("description"):
        yield format_meta(meta), "No public description found for this book, so I cannot produce a grounded summary.", cover_image
        return

    cfg = MODEL_OPTIONS[model_choice]
    client = cfg["client"]
    model = cfg["model"]

    prompt = build_user_prompt(meta)
    stream = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
        temperature=0,
        stream=True,
    )

    metadata_md = format_meta(meta)
    summary_text = ""

    # Show metadata + cover immediately, then stream summary text.
    yield metadata_md, "", cover_image

    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        if not delta:
            continue
        summary_text += delta
        yield metadata_md, summary_text, cover_image


def clear_all():
    return "", "", "", "", None

## 4) Gradio UI definition (book summarizer)

In [40]:
with gr.Blocks() as demo:
    gr.Markdown("## Week 2 Book Summarizer")
    gr.Markdown("Google Books lookup + model switching + streamed summaries")

    with gr.Row():
        title_input = gr.Textbox(label="Book title", placeholder="e.g. Atomic Habits")
        author_input = gr.Textbox(label="Author", placeholder="e.g. James Clear")

    with gr.Row():
        model_choice = gr.Dropdown(
            choices=list(MODEL_OPTIONS.keys()),
            value="OpenRouter: gpt-4.1-mini",
            label="Model",
        )

    with gr.Row():
        summarize_btn = gr.Button("Summarize", variant="primary")
        clear_btn = gr.Button("Clear")

    with gr.Row():
        metadata_output = gr.Markdown(label="Book metadata")
        cover_output = gr.Image(label="Book cover", height=320, interactive=False)

    summary_output = gr.Markdown(label="Summary")

    summarize_btn.click(
        fn=summarize_book_stream,
        inputs=[title_input, author_input, model_choice],
        outputs=[metadata_output, summary_output, cover_output],
    )

    author_input.submit(
        fn=summarize_book_stream,
        inputs=[title_input, author_input, model_choice],
        outputs=[metadata_output, summary_output, cover_output],
    )

    clear_btn.click(
        fn=clear_all,
        outputs=[title_input, author_input, metadata_output, summary_output, cover_output],
    )

## 5) Launch the app

In [ ]:
demo.launch(inbrowser=True)